In [36]:
import pandas as pd
import numpy as np

In [19]:
co_occ = pd.read_csv("cofreqs_terms_perBin_1d.txt", delimiter = "\t", header=None)

In [20]:
co_occ  = co_occ.rename(columns={0: "term1", 1:"term2", 2:"co_occur_freq"})

In [103]:
singlet = pd.read_csv("singlets_terms_perBin_1d.txt", delimiter = "\t", header=None)

In [104]:
singlet = singlet.rename(columns={0: "term", 1:"singleton_freq"})

In [96]:
singlet

,term,singleton_freq,subsample_rate
0,32208,312,0.999393
1,31537,784,0.999038
2,17507,1075,0.998873
3,34221,371,0.999338
4,16836,2846,0.998167
...,...,...,...
56590,43624,329,0.999377
56591,8732,17246,0.995487
56592,58447,249,0.999458
56593,42953,194,0.999521


In [24]:
intermed = co_occ.merge(singlet, left_on="term1", right_on = "term")

In [30]:
intermed = intermed.rename(columns={"singleton_freq": "term1_freq"}).drop(columns='term')

In [32]:
out_df = intermed.merge(singlet, left_on="term2", right_on = "term").rename(columns={"singleton_freq": "term2_freq"}).drop(columns='term')

In [42]:
out_df['PMI'] = np.log((out_df['co_occur_freq'] * len(singlet)) / (out_df['term1_freq'] * out_df['term2_freq']))
out_df['PPMI'] = np.where(out_df['PMI'] < 0, 0, out_df['PMI'])

In [106]:
singlet_list = singlet['term'].to_list()

In [108]:
from random import sample

In [146]:
def sample_terms(singlet_list, rate=0.01):
    
    num = round(len(singlet_list) * rate)
    
    return sample(singlet_list, num)

In [147]:
sampled_singlet = sample_terms(singlet_list)
print(len(sampled_singlet))

566


In [148]:
dataset = {}
i = 0
for singleton in sampled_singlet:
    if i%50 == 0:
        print(i)
    dataset[singleton] = out_df[out_df['term1']==singleton][['term2', 'PPMI', 'co_occur_freq']].values.tolist()
    i+=1

0
50
100
150
200
250
300
350
400
450
500
550


In [152]:
dataset

{28439: [[42852.0, 0.0, 132.0], [28439.0, 4.265333806412189, 795.0]],
 109459: [[109459.0, 6.328555404225261, 101.0]],
 13592: [[14110.0, 0.0, 147.0],
  [14379.0, 0.0, 107.0],
  [138182.0, 0.0, 231.0],
  [60091.0, 0.0, 116.0],
  [52314.0, 0.0, 103.0],
  [42852.0, 0.0, 309.0],
  [26789.0, 0.0, 103.0],
  [19748.0, 0.0, 362.0],
  [19116.0, 0.0, 154.0],
  [30025.0, 0.0, 113.0],
  [64980.0, 0.0, 116.0],
  [16582.0, 0.0, 107.0],
  [19861.0, 0.0, 102.0],
  [19097.0, 0.0, 106.0],
  [21092.0, 0.0, 108.0],
  [13592.0, 3.852766098986537, 1201.0],
  [86967.0, 0.0, 157.0],
  [43029.0, 0.09018083493761068, 151.0],
  [70476.0, 0.0, 252.0]],
 18574: [[20999.0, 0.0, 450.0],
  [20877.0, 0.0, 247.0],
  [20636.0, 0.0, 167.0],
  [20885.0, 0.0, 636.0],
  [21472.0, 0.0, 346.0],
  [20891.0, 0.0, 167.0],
  [20894.0, 0.0, 370.0],
  [21488.0, 0.0, 1867.0],
  [21365.0, 0.0, 248.0],
  [21245.0, 0.0, 171.0],
  [20669.0, 0.0, 1340.0],
  [21013.0, 0.0, 116.0],
  [98121.0, 0.0, 1007.0],
  [21265.0, 0.0, 195.0],
  [211

In [151]:
import pickle

with open('C://Users//wesle//Documents//sue_gtech//BD4H//BD4H-Project//data//sub_neighbors_dict_ppmi_per_Bin_1.pkl', 'wb') as handle:
    pickle.dump(dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)